<a href="https://colab.research.google.com/github/Beloveddream/RS/blob/main/Product2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

train_data = pd.read_csv('/content/gdrive/My Drive/Product/sant/train_update.csv')
test_data = pd.read_csv('/content/gdrive/My Drive/Product/sant/test_update.csv')

train_data = train_data.drop('origin',axis = 1)
test_data = test_data.drop(['item_1','item_2','item_3','item_4','item_5','item_6',
                          'item_7','item_8','item_9','item_10','item_11','item_12',
                          'item_13','item_14','item_15','item_16','item_17','item_18',
                          'item_19','item_20','item_21','item_22','item_23','item_24','origin'],axis =1)
train_data['item_22'] = train_data['item_22'].replace(np.nan, 0)
train_data['item_23'] = train_data['item_23'].replace(np.nan, 0)

train_data = train_data.sort_values(by=['user_id','date'],ascending=[True, False])
train_data = train_data.reset_index(drop = True)

train_data_test1 = train_data[['user_id','date','item_1','item_2','item_3',
                              'item_4','item_5','item_6','item_7','item_8',
                              'item_9','item_10','item_11','item_12','item_13',
                              'item_14','item_15','item_16','item_17','item_18',
                              'item_19','item_20','item_21','item_22','item_23','item_24']]

user_id_list = list(set(train_data_test1['user_id']))
user_id_list.sort()

col_name = list()
for num1 in range(1,25):
    col_name.append('item'+str(num1)+'_buy')

#对产品列进行处理，用户第一次出现的那行产品信息不变，后续日期行进行差分，差分结果为1表示用户在这个日期新购买了该产品，
#-1表示用户在这个日期卖出了该产品，0表示用户未对产品进行操作，将差分结果作为训练的目标标签
train_data_all = pd.DataFrame(columns = col_name)
for user_id_num in tqdm(user_id_list):
    train_data_one = train_data_test1[train_data_test1.user_id == user_id_num]
    train_data_one_temp = train_data_one.drop(['user_id','date'],axis=1)
    train_data_one_matrix = train_data_one_temp.values
    row_num = len(train_data_one_matrix)
    if row_num == 1:
        train_data_one_matrix = train_data_one_matrix
    else:
        for i in range(row_num-1): 
            train_data_one_matrix[i] = train_data_one_matrix[i+1]*(-1)+train_data_one_matrix[i]
    train_data_one_new = pd.DataFrame(train_data_one_matrix,columns=col_name)
    train_data_all = pd.concat([train_data_all, train_data_one_new],ignore_index = True)

100%|██████████| 39697/39697 [11:43<00:00, 56.46it/s]


In [3]:
train_data1 = pd.concat([train_data,train_data_all] ,axis =1)

train_data1 = train_data1.drop(['item_1','item_2','item_3',
                              'item_4','item_5','item_6','item_7','item_8',
                              'item_9','item_10','item_11','item_12','item_13',
                              'item_14','item_15','item_16','item_17','item_18',
                              'item_19','item_20','item_21','item_22','item_23','item_24'],axis =1)

train_data1.to_csv('train_data1.csv',index = 0)

In [4]:
train_data1.to_csv('/content/gdrive/My Drive/Product/sant/train_data1.csv',index = 0)

In [5]:
#去除日期标签，新增月标记，因为用户的产品购买存在一定的月份特征
train_data1['date']=pd.to_datetime(train_data1['date'],format='%Y-%m-%d')
train_data1['month_label'] = train_data1['date'].map(lambda x: x.month)

for num in col_name:
    train_data1[num] = train_data1[num].map(lambda x: 0 if x<0 else x)

train_data1['use_time'] = train_data1['use_time'].map(lambda x: 0 if x<0 else x)
train_data1 = train_data1.drop('date',axis =1)
test_data = test_data.drop('date',axis = 1)
test_data['month_label'] = 5

train_data_x = train_data1.drop(['item1_buy','item2_buy','item3_buy','item4_buy',
                             'item5_buy','item6_buy','item7_buy','item8_buy','item9_buy','item10_buy',
                             'item11_buy','item12_buy','item13_buy','item14_buy','item15_buy','item16_buy',
                             'item17_buy','item18_buy','item19_buy','item20_buy','item21_buy','item22_buy',
                             'item23_buy','item24_buy'],axis = 1)
train_data_y = train_data1[['item1_buy','item2_buy','item3_buy','item4_buy',
                             'item5_buy','item6_buy','item7_buy','item8_buy','item9_buy','item10_buy',
                             'item11_buy','item12_buy','item13_buy','item14_buy','item15_buy','item16_buy',
                             'item17_buy','item18_buy','item19_buy','item20_buy','item21_buy','item22_buy',
                             'item23_buy','item24_buy']]

train_data_x['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data_x,test_data],ignore_index = True,sort = False)
print(matrix.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561786 entries, 0 to 561785
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 561786 non-null  int64  
 1   user_country            561786 non-null  object 
 2   sex                     561786 non-null  object 
 3   age                     561786 non-null  int64  
 4   new_user_type           561786 non-null  float64
 5   user_seniority          561786 non-null  int64  
 6   user_type               561786 non-null  float64
 7   user_type_at_begin      561786 non-null  float64
 8   user_relation_at_begin  561786 non-null  object 
 9   live_country_type       561786 non-null  object 
 10  born_country_type       561786 non-null  object 
 11  user_join_channel       561786 non-null  object 
 12  province_code           561786 non-null  float64
 13  activity_type           561786 non-null  float64
 14  user_segmentation   

In [6]:
#对特征列进行Labelencoder
from sklearn.preprocessing import LabelEncoder

lbe_user_id = LabelEncoder()
matrix['user_id_lbe'] = lbe_user_id.fit_transform(matrix['user_id'])

cate_col = ['user_country','sex','new_user_type','user_type','user_type_at_begin','user_relation_at_begin',
           'live_country_type','born_country_type','user_join_channel','province_code','activity_type','user_segmentation',
           'month_label']

for name in cate_col:
    lbe_temp = LabelEncoder()
    matrix[name] = lbe_temp.fit_transform(matrix[name])

label_name =['user_country','sex','age','new_user_type','user_seniority',
            'user_type','user_type_at_begin','user_relation_at_begin','live_country_type',
            'born_country_type','user_join_channel','province_code','activity_type',
            'user_segmentation','use_time','month_label']

#对特征列进行类型变换以减少内存消耗
for name in label_name:
    matrix[name] = matrix[name].astype('int16')
    
matrix['user_income'] = matrix['user_income'].astype('int32')
matrix['user_id_lbe'] = matrix['user_id_lbe'].astype('int32')

train_data_x = matrix[matrix.origin == 'train']
test_data = matrix[matrix.origin == 'test']

print(train_data_x.describe())

train_data_x.to_csv('train_data_x.csv',index = 0)
train_data_y.to_csv('train_data_y.csv',index = 0)
test_data.to_csv('test_data.csv',index = 0)

            user_id   user_country  ...    month_label    user_id_lbe
count  5.326240e+05  532624.000000  ...  532624.000000  532624.000000
mean   8.248920e+05      15.018585  ...       4.722275   18353.212133
std    4.330362e+05       0.867818  ...       3.602370   10688.213301
min    1.607400e+04       0.000000  ...       0.000000       0.000000
25%    4.397070e+05      15.000000  ...       2.000000    9086.000000
50%    9.212120e+05      15.000000  ...       4.000000   18449.000000
75%    1.194005e+06      15.000000  ...       8.000000   27354.000000
max    1.548191e+06      42.000000  ...      11.000000   39696.000000

[8 rows x 19 columns]


In [7]:
train_data_x.to_csv('/content/gdrive/My Drive/Product/train_data_x.csv',index = 0)
train_data_y.to_csv('/content/gdrive/My Drive/Product/train_data_y.csv',index = 0)
test_data.to_csv('/content/gdrive/My Drive/Product/test_data.csv',index = 0)

In [8]:
train_data_x = pd.read_csv('/content/gdrive/My Drive/Product/train_data_x.csv')
train_data_y = pd.read_csv('/content/gdrive/My Drive/Product/train_data_y.csv')
test_data = pd.read_csv('/content/gdrive/My Drive/Product/test_data.csv')

train_data_x = train_data_x.drop(['origin','user_id'],axis = 1)
all_data = pd.concat([train_data_x,train_data_y],axis = 1)

test_data = test_data.drop('origin',axis = 1)
test_data1 = test_data.drop('user_id',axis = 1)

#all_data.to_csv('all_data.csv',index=0)
#将24个目标标签列整合成1列，如果用户购买了某一产品，则在标签列显示相应产品序号
all_data1 = all_data.set_index(['user_country', 'sex', 'age', 'new_user_type', 'user_seniority', 'user_type', 
                                'user_type_at_begin', 'user_relation_at_begin', 'live_country_type', 
                                'born_country_type', 'user_join_channel', 'province_code', 'activity_type', 
                                'user_segmentation', 'use_time', 'user_income', 'month_label', 'user_id_lbe' ])
all_data1 = all_data1.stack().reset_index()
#print(all_data1)

all_data1['level_18'] = all_data1['level_18'].map({'item1_buy': 0, 'item2_buy': 1,'item3_buy': 2,
                                                  'item4_buy': 3, 'item5_buy': 4,'item6_buy': 5,
                                                  'item7_buy': 6, 'item8_buy': 7,'item9_buy': 8,
                                                  'item10_buy': 9, 'item11_buy': 10,'item12_buy': 11,
                                                  'item13_buy': 12, 'item14_buy': 13,'item15_buy': 14,
                                                  'item16_buy': 15, 'item17_buy': 16,'item18_buy': 17,
                                                  'item19_buy': 18, 'item20_buy': 19,'item21_buy': 20,
                                                  'item22_buy': 21, 'item23_buy': 22,'item24_buy': 23})

#print(all_data1)

all_data1['buy'] = all_data1['level_18'].mul(all_data1[0])
#print(all_data1[all_data1.buy != 0])

all_data2 = all_data1[all_data1.buy != 0]

all_data2 = all_data2.drop(['level_18',0],axis =1)
#print(all_data2)

all_data2['buy'] = all_data2['buy'].astype('int32')

#print(all_data2['buy'].value_counts())

test_data2 = test_data1.values
#print(test_data2)

all_data3 = all_data2.drop(['buy'],axis=1).values

all_data4 = all_data2['buy'].values
#print(all_data4)

#进行Xgboost的预测
from sklearn.model_selection import train_test_split
import xgboost as xgb

X = all_data3
Y = all_data4

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
#加载numpy的数组到DMatrix对象
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix( X_test, label=y_test)
#1.训练模型
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'mlogloss'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 15
#param['silent'] = 0
#param['nthread'] = 4
param['num_class'] = 24
param['lamdba'] = 2
#param['colsample_bytree'] = 0.8
#param['subsample'] = 0.9
 
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 10000
bst = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=100, verbose_eval=25 );

test_data_input = xgb.DMatrix(test_data2)
pred = bst.predict( test_data_input );
print (pred)

[0]	train-mlogloss:3.13894	test-mlogloss:3.14264
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 100 rounds.
[25]	train-mlogloss:2.5739	test-mlogloss:2.66398
[50]	train-mlogloss:2.26553	test-mlogloss:2.42834
[75]	train-mlogloss:2.0515	test-mlogloss:2.27557
[100]	train-mlogloss:1.88974	test-mlogloss:2.16799
[125]	train-mlogloss:1.7625	test-mlogloss:2.08912
[150]	train-mlogloss:1.65953	test-mlogloss:2.02956
[175]	train-mlogloss:1.57592	test-mlogloss:1.98382
[200]	train-mlogloss:1.506	test-mlogloss:1.94899
[225]	train-mlogloss:1.44666	test-mlogloss:1.92263
[250]	train-mlogloss:1.39756	test-mlogloss:1.90182
[275]	train-mlogloss:1.3548	test-mlogloss:1.88584
[300]	train-mlogloss:1.31693	test-mlogloss:1.87349
[325]	train-mlogloss:1.28487	test-mlogloss:1.86474
[350]	train-mlogloss:1.2567	test-mlogloss:1.85841
[375]	train-mlogloss:1.23054	test-mlogloss:1.85408
[400]	train-mlogloss:1.2081	test-mlogloss:1.

In [9]:
submit = pd.DataFrame(pred,columns = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
                        'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
                        'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
                        'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
                        'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
                        'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                        'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
                        'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1'])

In [10]:
submit

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0.003151,0.003190,0.102433,0.003379,0.038209,0.003171,0.003237,0.010292,0.018934,0.003354,0.003521,0.038046,0.023194,0.004167,0.003349,0.009588,0.003408,0.011267,0.115163,0.006136,0.003275,0.150671,0.167407,0.271455
1,0.002775,0.002809,0.087507,0.002973,0.017369,0.002793,0.002851,0.009600,0.020965,0.005636,0.003101,0.009745,0.032843,0.008348,0.002950,0.005534,0.003066,0.008233,0.156634,0.008389,0.002885,0.096088,0.125454,0.381451
2,0.003781,0.003828,0.078278,0.004055,0.044041,0.003806,0.003884,0.117019,0.037769,0.003970,0.004852,0.038213,0.018455,0.007228,0.004019,0.008748,0.004724,0.008201,0.091481,0.008664,0.003930,0.092646,0.154109,0.254298
3,0.002488,0.002518,0.720325,0.002668,0.023359,0.002504,0.003468,0.002502,0.002515,0.002612,0.002780,0.004160,0.003841,0.002583,0.002632,0.002635,0.002727,0.014307,0.002740,0.003188,0.002523,0.043848,0.068778,0.078299
4,0.002978,0.003015,0.148244,0.003194,0.058630,0.002997,0.003059,0.009324,0.031621,0.003127,0.003328,0.016362,0.015313,0.005742,0.003165,0.003156,0.003331,0.012597,0.065072,0.006372,0.003940,0.181425,0.130997,0.283011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29157,0.000964,0.000976,0.133337,0.001034,0.007615,0.000970,0.001032,0.000969,0.000974,0.001012,0.001033,0.001068,0.002277,0.001001,0.001053,0.001007,0.000983,0.003509,0.002566,0.001131,0.000977,0.021774,0.048132,0.764607
29158,0.003398,0.003440,0.216257,0.003644,0.022616,0.003420,0.003800,0.003417,0.003435,0.003568,0.003641,0.003827,0.005408,0.003519,0.003595,0.003549,0.003464,0.009035,0.011402,0.003649,0.003445,0.007544,0.010521,0.660406
29159,0.000839,0.000849,0.098271,0.000899,0.006817,0.000844,0.000898,0.000844,0.000848,0.000881,0.000899,0.000929,0.001981,0.000871,0.000916,0.000876,0.000855,0.003353,0.002233,0.000984,0.000850,0.019238,0.036916,0.817108
29160,0.002150,0.002176,0.421920,0.002305,0.012742,0.002163,0.002302,0.002162,0.002173,0.002257,0.002303,0.002379,0.003281,0.002226,0.002347,0.002248,0.002191,0.008091,0.007213,0.002522,0.002180,0.006337,0.006982,0.497349


In [11]:
submit.to_csv('/content/gdrive/My Drive/Product/submit_small.csv',index = 0)

In [12]:
print(submit.loc[0,:].to_list())

[0.0031513150315731764, 0.0031899199821054935, 0.1024334505200386, 0.0033792282920330763, 0.038209497928619385, 0.003171465126797557, 0.003236575983464718, 0.010292001068592072, 0.01893390528857708, 0.0033540071453899145, 0.0035210619680583477, 0.03804628551006317, 0.023193949833512306, 0.004167178645730019, 0.003349329112097621, 0.00958787277340889, 0.0034084822982549667, 0.011266996152698994, 0.11516274511814117, 0.006136270239949226, 0.003275312017649412, 0.15067139267921448, 0.16740652918815613, 0.27145522832870483]


In [15]:
test_data = pd.read_csv('/content/gdrive/My Drive/Product/test_data.csv')
submit_list = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
                'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
                'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
                'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
                'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
                'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']
submit_all = pd.DataFrame(columns = ['ncodpers','added_products'])
submit_all['ncodpers'] = test_data['user_id']
print(submit_all)

       ncodpers added_products
0        657788            NaN
1        657794            NaN
2        657770            NaN
3        658056            NaN
4        657869            NaN
...         ...            ...
29157   1166847            NaN
29158   1166605            NaN
29159   1166829            NaN
29160   1166827            NaN
29161   1166781            NaN

[29162 rows x 2 columns]


In [16]:
#由于预测结果是全部产品的购买概率，每个用户选取概率最高的7个产品依次作为提交结果
for num_1 in tqdm(range(len(submit))):
    temp_list1 = submit.loc[num_1,:].to_list()
    temp_list1 = np.array(temp_list1)
    temp_1 = np.argsort(-temp_list1)
    submit_one = ''
    for num_2 in range(7):
        submit_one += submit_list[temp_1[num_2]] + ' '
    submit_all.loc[ num_1 , 'added_products'] = submit_one

100%|██████████| 29162/29162 [00:35<00:00, 823.91it/s]


In [17]:
print(submit_all)

       ncodpers                                     added_products
0        657788  ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...
1        657794  ind_recibo_ult1 ind_tjcr_fin_ult1 ind_nom_pens...
2        657770  ind_recibo_ult1 ind_nom_pens_ult1 ind_ctop_fin...
3        658056  ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...
4        657869  ind_recibo_ult1 ind_nomina_ult1 ind_cco_fin_ul...
...         ...                                                ...
29157   1166847  ind_recibo_ult1 ind_cco_fin_ult1 ind_nom_pens_...
29158   1166605  ind_recibo_ult1 ind_cco_fin_ult1 ind_cno_fin_u...
29159   1166829  ind_recibo_ult1 ind_cco_fin_ult1 ind_nom_pens_...
29160   1166827  ind_recibo_ult1 ind_cco_fin_ult1 ind_cno_fin_u...
29161   1166781  ind_recibo_ult1 ind_cco_fin_ult1 ind_tjcr_fin_...

[29162 rows x 2 columns]


In [18]:
submit_all.to_csv('/content/gdrive/My Drive/Product/sub_samll2.csv',index=0)